In [44]:
using NCDatasets
using DIVAnd
using PyPlot
using Dates
using Glob
using Test
const plt = PyPlot
include("./config.jl");
include("./ME4OH.jl")
doplot = false

false

## Create list of netCDF files
The files with the "nonan" extension were moved to another directory.

In [40]:
datafilelist = get_filelist(datadir, timeperiod)

2-element Vector{String}:
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197901.update.nc"
 "/home/ctroupin/data/ME4OH/data/" ⋯ 55 bytes ⋯ "f.profiles.g10.197902.update.nc"

## Experiment B
We perform purely 2D interpolations by looping on the monthly periods and the depth levels.     
The land-sea mask is created on the 3 specified depth levels.

In [9]:
depth_dohc = [depthlayer1[end], depthlayer2[end], depthlayer3[end]]

3-element Vector{Float64}:
  286.6
  685.9
 1985.3

In [10]:
xi, yi, mask_dohc = DIVAnd.load_mask(bathyfile, true, longrid, latgrid, depth_dohc);        
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);

### Prepare the time grid 
According to the time period specified in `config.jl`.

In [22]:
timegrid1 = get_timegrid(timeperiod1);

## Perform analysis
### Create the netCDF files that will store the results
The output directory is specified in `config.jl`.

In [114]:
outputfilelist = []
for depthlayer in [depthlayer1, depthlayer2, depthlayer3]
    fname = make_fname(timeperiod1, depthlayer, "A")
    outputfile = joinpath(outputdir, fname)
    isfile(outputfile) ? rm(outputfile) : @debug("OK")
    create_netcdf_results(outputfile, longrid, latgrid, timegrid1)
    
    # Add the residual variables in the netCDF
    add_residuals(outputfile, datafilelist)
    
    # Add file to the list
    push!(outputfilelist, outputfile)
end

### Loop on the files and the depths

In [115]:
for (ntime, datafile) in enumerate(datafilelist)
    
    # Read observations
    lon, lat, dates, depth, T, S, dohc = read_profile(datafile)
    
    # Modify coordinates
    lon[lon .< 20.] .+= 360
    
    # Loop on the 3 depth levels
    for ii = 1:1
        
        # Select good values (no NaNs)
        obsval = dohc[ii,:]
        goodval = findall(.!isnan.(obsval));
        ngood = length(goodval)
        
        # Perform interpolation
        fi, s = DIVAndrun(mask_dohc[:,:,ii], (pm, pn), (xi, yi), (Float64.(lon[goodval]), 
        Float64.(lat[goodval])), Float64.(obsval[goodval] .- mean(obsval[goodval])), 
        (5., 5.), 10.; moddim=[1,0])
        
        # Compute residuals
        dataresidual = DIVAnd_residual(s, fi)
        
        # Write field and residual in the netCDF
        NCDataset(outputfilelist[ii], "a") do ds
            ds["dohc"][:,:,ntime] = fi
            ds["obslon"][1:ngood,ntime] = lon[goodval]
            ds["obslat"][1:ngood,ntime] = lat[goodval]
            ds["obstime"][1:ngood,ntime] = dates[goodval]
            ds["dohc_residuals"][1:ngood,ntime] = dataresidual
        end
        
        # Make plot
        if doplot
            fig = plt.figure(figsize=(15, 8))
            ax = plt.subplot(121)
            ax.scatter(lon[goodval], lat[goodval], s=3, c=obsval[goodval])
            ax.set_xlim(20., 380.)
            ax = plt.subplot(122)
            ax.pcolormesh(longrid, latgrid, fi' .+ mean(obsval[goodval]))
            ax.set_xlim(20., 380.)
            plt.show()
        end
    end
end

## Temperature analysis

In [42]:
datafile = datafilelist[1]

"/home/ctroupin/data/ME4OH/data/en4.1.1/1979-2014/full/update/ofam3-jra55.all.EN.4.1.1.f.profiles.g10.197901.update.nc"

In [45]:
lon, lat, dates, depth, T, S, dohc = ME4OH.read_profile(datafile)
obslon, obslat, obsdates, obsdepth, T, S = ME4OH.vectorize_obs(lon, lat, dates, depth, T, S)

(Float32[165.55, 165.55, 165.55, 165.55, 165.55, 165.55, 165.55, 165.55, 165.55, 165.55  …  31.25, 31.25, 31.25, 31.25, 31.25, 31.25, 31.25, 31.25, 31.25, 31.25], Float32[-74.75, -74.75, -74.75, -74.75, -74.75, -74.75, -74.75, -74.75, -74.75, -74.75  …  74.95, 74.95, 74.95, 74.95, 74.95, 74.95, 74.95, 74.95, 74.95, 74.95], [DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00"), DateTime("1979-01-07T00:00:00")  …  DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00"), DateTime("1979-01-16T00:00:00")], Float32[2.

In [47]:
dbinfo = diva3d((longrid, latgrid), (obslon, obslat, obsdepth)x,value,len,epsilon2,filename,varname)

search: diva3d DIVAnd_filter3



```
dbinfo = diva3d(xi,x,value,len,epsilon2,filename,varname)
```

Create a 3D analysis (or a series of 3D analysis) with DIVAnd using the observations `value` (vector) at the locations `x` (tuple of vectors) onto the regular grid defined by the vectors `xi` using the scaled observational error variance `epsilon2` and the correlation length `len`. The result will be saved in the netCDF file `filename` under the variable `varname`.

## Inputs

  * `xi`: tuple with n elements. Every element represents a coordinate of the final grid on which the observations are interpolated
  * `x`: tuple with n elements. Every element represents a coordinate of the observations
  * `value`: value of the observations
  * `len`: tuple with n elements. Every element represents the correlation length.  If `fitcorrlen` is `false` (default), the correlation length should be expressed in meters.  If `fitcorrlen` is `true`, then `len` can be the empty tuple `()` or a tuple containing  3 arrays of normalized (or relative) correlation lengths which will be multiplied by the  horizontal and vertical correlation lengths. In the case where `fitcorrlen` is `true`  and `len` is provided, the elements of the tuple `len` are adimensional and mostly of the  order of 1. Where the elements of `len` are less than 1, the correlation length (obtained via  fitting, DIVAnd.fithorzlen and DIVAnd.fitvertlen) is effectively decreased and where it is larger  than 1 it is increased. This is useful for example to decrease the correlation length locally  near steep topography. It is advised to check the range of the scaled correlation length which  is printed on the screen while running DIVAnd. The fitted values are also returned in the structure `dbinfo`.  The correlation length fitting can produce irrealistic results for inhomogenous data coverage.
  * `epsilon2`: error variance of the observations (normalized by the error variance of the background field).

`epsilon2` can be a scalar (all observations have the same error variance and their errors are decorrelated), a vector (all observations can have a different error variance and their errors are decorrelated) or a matrix (all observations can have a different error variance and their errors can be correlated). If `epsilon2` is a scalar, it is thus the *inverse of the signal-to-noise ratio*.

  * `filename`: the output netCDF filename.
  * `varname`: the name of the variable (used in the netCDF file).

## Optional input arguments:

  * `bathname`: path to the netCDF bathymetry (default `../../DIVAnd-example-data/Global/Bathymetry/gebco_30sec_16.nc` relative to this source file)
  * `bathisglobal`: true (default) is the bathymetry is a global data set
  * `plotres`: call-back routine for plotting ((timeindex,sel,fit,erri) -> nothing)
  * `timeorigin`: time origin (default `DateTime(1900,1,1,0,0,0)`)
  * `moddim`: modulo for cyclic dimension (vector with n elements).    Zero is used for non-cyclic dimensions. Halo points should    not be included for cyclic dimensions. For example if the first dimension    is cyclic, then the grid point corresponding to `mask[1,j]` should be    between `mask[end,1]` (left neighbor) and `mask[2,j]` (right neighbor). The default is [0,0,0],
  * `zlevel`: `:surface` (default) for surface analysis and `:floor` for analysis from the bottom floor.
  * `ncvarattrib`: dictionary of netCDF variable attributes.
  * `ncglobalattrib`: dictionary of netCDF global attributes.
  * `transform`: Anamorphosis transformation function (default: `Anam.notransform()`).
  * `fitcorrlen`: true if the correlation length is determined from the observation (default `false`).    Note that the parameter `len` is interpreted differently when `fitcorrlen` is set to `true`.
  * `fithorzcorrlen`: true if the horizontal correlation length is determined from the observation (default: the value of `fitcorrlen`)    Note that the parameter `len` is interpreted differently when `fithorzcorrlen` is set to `true`.
  * `fitvertcorrlen`: true if the vertical correlation length is determined from the observation (default: the value of `fitcorrlen`)    Note that the parameter `len` is interpreted differently when `fitvertcorrlen` is set to `true`.
  * `fithorz_param`: dictionary with additional optional parameters for `fithorzlen`, for example: `Dict(:smoothz => 200., :searchz => 50.)`.
  * `fitvert_param`: dictionary with additional optional parameters for `fitvertlen`.
  * `distfun`: function to compute the distance (default `(xi,xj) -> DIVAnd.distance(xi[2],xi[1],xj[2],xj[1])`).
  * `mask`: if different from `nothing`, then this mask overrides land-sea mask based on the bathymetry

(default `nothing`).

  * `background`: if different from `nothing`, then this parameter allows one

to load the background from a call-back function (default `nothing`). The call-back functions has the parameters `(x,n,trans_value,trans)` where `x` represent the position of the observations, `n` the time index, `trans_value`, the observations (possibly transformed) and `trans` the transformation function. The output of this function is the gridded background field and the observations minus the background field. See also `DIVAnd.backgroundfile`.

  * `background_epsilon2_factor`: multiplication for `epsilon2` when computing a  vertical profile as a background estimate (default: computed internally based on the amount of data). This parameter is not used  when the parameter `background` or `background_lenz` is provided.
  * `background_lenz`: vertical correlation for background computation (default 20 m). This parameter is not used  when the parameter `background` is provided.
  * `background_len`: deprecated option replaced by `background_lenz`.
  * `filterbackground`: number of iterations to filter the background profile (default 0, no filtering)
  * `memtofit`: keyword controlling how to cut the domain depending on the memory   remaining available for inversion. It is not the total memory (default 3). Use a large value (e.g. 100) to force the   usage for the more efficient direct solver if you are not limited by the amount of RAM memory.
  * `minfield`: if the analysed field is below `minfield`, its value is replace by `minfield` (default -Inf, i.e. no substitution is done).
  * `maxfield`: if the analysed field is above `maxfield`, its value is replace by `maxfield` (default +Inf, i.e. no substitution is done).
  * `saveindex`: controls if just a subset of the analysis should be saved to   the netCDF file. Per default, `saveindex` is `(:,:,:)` (corresponding to   longitude, latitude and depth indices) meaning that everything is saved.   If however, for example the first layer should not be saved then `saveindex`   should be `(:,:,2:length(depthr))` where `depthr` is the 3rd element of `xi`.
  * `niter_e`: Number of iterations to estimate the optimal scale factor of  `epsilon2` using Desroziers et al. 2005 (doi: 10.1256/qj.05.108). The default   is 1 (i.e. no optimization is done).
  * `coeff_derivative2` (vector of 3 floats): for every dimension where this value is non-zero, an additional term is added to the cost function penalizing the second derivative. A typical value of this parameter is `[0.,0.,1e-8]`.
  * `surfextend`: create an additional layer on top of the surface layer so that the effective background error variance is more similar to the deep ocean.  `false` is the default value.
  * `stat_per_timeslice` (default false): if true, then the residual values (and possibly qcvalues) are also returned by time slices which is useful if the time slices overlap (see example below).
  * `error_thresholds` (default `[("L1", 0.3), ("L2", 0.5)]`). This is a list of tuples with the applied error thresholds and the variable names suffixes. If the variable is named e. g. `"Salinity"`, then the variables `"Salinity_L1"` (resp.  `"Salinity_L2"`) will be created where the analysis is masked if the relative error exceeds 0.3 (resp. 0.5).
  * `divamethod`: `DIVAndgo` (default) or `DIVAndrun`

Any additional keywoard arguments understood by `DIVAndgo`/`DIVAndrun` can also be used here (e.g. velocity constrain)

The output is a dictionary with the followings keys:

  * `:residuals`: the difference between the observations and the analysis (interpolated linearly to the

location of the observations). The residual is `NaN` if the observations are not within the domain as defined by the mask and the coordinates of the observations `x`.

  * `:qcvalues`: quality control scores (if activated)

## Example

Example on how to aggredate the residuals per time slice and to retain the maximum residual:

```julia
selection_per_timeslice = dbinfo[:selection_per_timeslice]
residuals_per_timeslice = dbinfo[:residuals_per_timeslice]
selection_per_timeslice = dbinfo[:selection_per_timeslice]

max_residuals = fill(-Inf,length(value))
for n = 1:length(selection_per_timeslice)
   sel = selection_per_timeslice[n]
   max_residuals[sel] = max.(max_residuals[sel],residuals_per_timeslice[n])
end
```

!!! note
    At all vertical levels, there should at least one sea point. The function assumes a spherical Earth with a radius equal to the mean Earth radius.



In [48]:
?DIVAnd.load_mask

```
xi,yi,mask = load_mask(bath_name,isglobal,xi,yi,level::Number)
```

Generate a land-sea mask based on the topography from the NetCDF file `bathname`. The parameter `isglobal` is true if the NetCDF file covers the whole globe and thus the last longitude point can be considered to be right next to the first longitude point. `xi` and `yi` are vectors containing the longitude and latitude grid onto which the bathymetry should be interpolated.

**Convention:** in the water, `level` is positive and in the air `level` is negative.
